In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
file_path = '/content/drive/My Drive/test_data.csv'
data = pd.read_csv(file_path)

In [ ]:
data.head()

,document,summary,id
0,"Prison Link Cymru had 1,099 referrals in 2015-...","There is a ""chronic"" need for more housing for...",38264402
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am...",34227252
2,"Jordan Hill, Brittany Covington and Tesfaye Co...",Four people accused of kidnapping and torturin...,38537698
3,The 48-year-old former Arsenal goalkeeper play...,West Brom have appointed Nicky Hammond as tech...,36175342
4,Restoring the function of the organ - which he...,The pancreas can be triggered to regenerate it...,39070183


In [3]:
documents = data['document']
summaries = data['summary']

In [4]:
specified_documents = documents[:5]

In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BertTokenizer, EncoderDecoderModel
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [10]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
pegasus_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
pegasus_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [9]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [11]:
def summarize_with_bart(article):
    inputs = bart_tokenizer("summarize : " + article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_with_bert(article):
    inputs = bert_tokenizer("summarize : " + article, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = bert_model.generate(inputs["input_ids"], decoder_start_token_id=bert_model.config.decoder.pad_token_id, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bert_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_with_t5(article):
    inputs = t5_tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = t5_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [5]:
cleaned_document = [doc.replace('\n', '') for doc in specified_documents]

In [ ]:
cleaned_document

list

In [14]:
bart_summaries = [summarize_with_bart(article) for article in cleaned_document]
bert_summaries = [summarize_with_bert(article) for article in cleaned_document]
t5_summaries = [summarize_with_t5(article) for article in cleaned_document]

In [15]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b1ea4d8fae05ec3675804fecb99ba7ed0391dc4c5d7e744b9500d5207dc885de
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [17]:
from rouge_score import rouge_scorer

def calculate_rouge_scores(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

    rouge1_precision = rouge1_recall = rouge1_f1 = 0
    rougeL_precision = rougeL_recall = rougeL_f1 = 0

    for ref, hyp in zip(reference, hypothesis):
        scores = scorer.score(ref, hyp)
        rouge1_precision += scores['rouge1'].precision
        rouge1_recall += scores['rouge1'].recall
        rouge1_f1 += scores['rouge1'].fmeasure
        rougeL_precision += scores['rougeL'].precision
        rougeL_recall += scores['rougeL'].recall
        rougeL_f1 += scores['rougeL'].fmeasure

    num_summaries = len(reference)
    rouge1_precision /= num_summaries
    rouge1_recall /= num_summaries
    rouge1_f1 /= num_summaries
    rougeL_precision /= num_summaries
    rougeL_recall /= num_summaries
    rougeL_f1 /= num_summaries

    return {
        "ROUGE-1 Precision": rouge1_precision,
        "ROUGE-1 Recall": rouge1_recall,
        "ROUGE-1 F1 Score": rouge1_f1,
        "ROUGE-L Precision": rougeL_precision,
        "ROUGE-L Recall": rougeL_recall,
        "ROUGE-L F1 Score": rougeL_f1,
    }


In [24]:
reference = summaries[:5]
bart_rouge_scores = calculate_rouge_scores(reference, bart_summaries[:5])
t5_rouge_scores = calculate_rouge_scores(reference, t5_summaries[:5])
bert_rouge_scores = calculate_rouge_scores(reference,bert_summaries[:5])

In [25]:
print("bart rouge scores \n")
print(bart_rouge_scores)

print("bert rouge scores \n")
print(bert_rouge_scores)

print("t5 rouge scores \n")
print(t5_rouge_scores)

bart rouge scores 

{'ROUGE-1 Precision': 0.1576714865311129, 'ROUGE-1 Recall': 0.3745843989769821, 'ROUGE-1 F1 Score': 0.2214167573399298, 'ROUGE-L Precision': 0.10155926153071974, 'ROUGE-L Recall': 0.24187979539641943, 'ROUGE-L F1 Score': 0.14270669253688206}
bert rouge scores 

{'ROUGE-1 Precision': 0.16770515544492354, 'ROUGE-1 Recall': 0.3443414322250639, 'ROUGE-1 F1 Score': 0.22404583472733358, 'ROUGE-L Precision': 0.09436400635331388, 'ROUGE-L Recall': 0.19533248081841434, 'ROUGE-L F1 Score': 0.1263579513772722}
t5 rouge scores 

{'ROUGE-1 Precision': 0.17948815684985897, 'ROUGE-1 Recall': 0.37202685421994885, 'ROUGE-1 F1 Score': 0.23851694915254235, 'ROUGE-L Precision': 0.11765676923123732, 'ROUGE-L Recall': 0.24494884910485934, 'ROUGE-L F1 Score': 0.15658705701078582}
